In [1]:
%pylab inline
%precision 6

Populating the interactive namespace from numpy and matplotlib


'%.6f'

In [2]:
import pandas as pd
import sklearn as skl
import sklearn
from keras.utils import np_utils

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
#you may download these functions from https://github.com/Apogentus/common
from common.serialization import pickle_load, pickle_save
from common.classes.Struct import Struct
from common.feature_transformations import get_one_hot_encoding
from common.functions import all_nums

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from collections import Counter

In [5]:
pd.options.display.max_colwidth=100
np.set_printoptions(linewidth=140,edgeitems=10)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
rcParams['figure.figsize'] = (8.0, 5.0)

In [6]:
Z_train = pd.read_csv('all/train.csv', skipinitialspace=True, sep=',', encoding='latin', index_col='PostId')

In [7]:
Z_test = pd.read_csv('all/test.csv', skipinitialspace=True, sep=',', encoding='latin', index_col='PostId')

In [8]:
Z_train.head()

,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus
PostId,,,,,,,,,,,,,,
0,10/31/2011 12:35:12,993291,10/13/2011 09:56:48,16,0,Compiler error 3921? Following a tutorial and can't work out what is causing this error,I am following a tutorial on Function x <http://www.functionx.com/vcnet/xml/readwrite.htm>. I h...,.net,xml,visual-c++,NaN,NaN,11/19/2011 23:40:06,not a real question
1,05/07/2009 15:19:43,82479,03/25/2009 04:27:50,169,8,Is the CareerCup Book Worth the money?,I just couldnt find a decent review of this book http://www.careercup.com/ \r\nHas anyone in thi...,books,career-development,interview-questions,NaN,NaN,01/31/2012 13:10:04,not constructive
3,12/14/2011 11:16:33,1078847,12/03/2011 07:32:14,1,0,Mailing with Java,I am planning to develop a web application in java using struts/servlets/jsp technology.\r\n\r\n...,java,javascript,servlets,dojo,javamail,12/14/2011 15:25:39,not a real question
5,11/22/2011 04:33:54,532638,12/06/2010 16:03:41,98,0,Ctrl+A doesn't work in Firefox for multi select,If there is a multi select control and you press Ctrl+A in Firefox to select all values then it ...,firefox,multi-select,NaN,NaN,NaN,11/22/2011 22:46:24,off topic
6,10/02/2011 21:01:24,975913,10/02/2011 20:54:48,1,0,How to detect this type of array value use php,"I have a array value bellow\r\n\r\n[ [[6,'SPAIN LA LIGA',0],[[843188,'RCD Espanyol','Real Madrid...",php,json,multidimensional-array,NaN,NaN,10/02/2011 22:13:04,not a real question


In [9]:
from gensim.models.keyedvectors import KeyedVectors
import gensim.downloader as api
# w2v_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


w2v_model1 = api.load("fasttext-wiki-news-subwords-300")
# w2v_model2 = api.load("glove-wiki-gigaword-300")

In [10]:
word2vec_size = 300

In [11]:
def vectorise(doc: str) -> np.array:
    """
    Identify the vector values for each word in the given document
    :param doc: a document to vectorise
    :return: a mean vector of word2vec vectors of the document
    """
    if not isinstance(doc, str):
        return numpy.zeros(shape=(word2vec_size,))
    doc = doc.lower()
    words = [w for w in doc.split(' ')] # if w not in stop_words
    word_vecs1 = []
    # word_vecs2 = []
    for word in words:
        try:
            vec1 = w2v_model1[word]
            # vec2 = w2v_model2[word]
            
            word_vecs1.append(vec1)
            # word_vecs2.append(vec2)
        except KeyError:
            # Ignore, if the word doesn't exist in the vocabulary
            pass
    if not word_vecs1:
        word_vecs1 = [numpy.zeros(shape=(word2vec_size,))]
#     if not word_vecs2:
#         word_vecs2 = [numpy.zeros(shape=(word2vec_size,))]
        
    # Assuming that document vector is the mean of all the word vectors
    #vector = np.concatenate([np.mean(word_vecs1, axis=0), np.mean(word_vecs2, axis=0)], axis=0)
    vector = np.mean(word_vecs1, axis=0)
    return vector

In [12]:
# Cast to datetime
Z_train['PostCreationDate'] = pd.to_datetime(Z_train['PostCreationDate'])
# Z_emb['PostCreationDate'] = pd.to_datetime(Z_emb['PostCreationDate'])
Z_train['PostClosedDate'] = pd.to_datetime(Z_train['PostClosedDate'])
# Z_emb['PostClosedDate'] = pd.to_datetime(Z_emb['PostClosedDate'])
Z_test['PostCreationDate'] = pd.to_datetime(Z_test['PostCreationDate'])
Z_train['OwnerCreationDate'] = pd.to_datetime(Z_train['OwnerCreationDate'])
# Z_emb['OwnerCreationDate'] = pd.to_datetime(Z_emb['OwnerCreationDate'])
Z_test['OwnerCreationDate'] = pd.to_datetime(Z_test['OwnerCreationDate'])

In [13]:
from keras.preprocessing.text import Tokenizer
from itertools import chain
from pandas import Series
import re
regex = r"[A-Z][\w\s\d\,\'\;\:\"\(\)\[\]\%\$\!\?]*(\.)"

tokenizer_title = Tokenizer()
tokenizer_title.fit_on_texts(list(chain(Z_train['BodyMarkdown'].values, Z_train['Title'].values,
                       #                 Z_emb['BodyMarkdown'].values, Z_emb['Title'].values,
                        Z_test['BodyMarkdown'].values, Z_test['Title'].values)))

tag_keys = ['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5']
tags_count = []
for tag in tag_keys:
    for x in list(Z_train[tag]):
        tags_count.append(x)
for tag in tag_keys:
    for x in list(Z_test[tag]):
        tags_count.append(x)
tags_count = Series(tags_count).value_counts()

def tag_weight(tag):
    if isinstance(tag, float):
        return 0
    else:
        return tags_count[tag]
    
def tags_weight(row):
    tags_sum = 0
    for tag in tag_keys:
        tags_sum += tag_weight(row[tag])
    return tags_sum

keywords = ['class', 'if', 'else', 'int', 'float', 'string', 'String', 'var', 
            'val', 'namespace', 'public', 'private', 'null', 'NaN', 'nan',
            'while', 'for', 'boolean', 'try', 'catch', 'byte', 'break', 'None',
            'double', '[', ']', '{', '}', '->', '=>', 'lambda', 'def', 'fun', 
            'enum', 'float', 'interface', 'return', 'char', 'yield', 'void', 
            'func', 'defer', 'import', 'struct', 'switch', 'static', 'sizeof', 
            'sizeOf', 'typedef', 'const', '==', '===', '!=', '>=', '<=', '+=',
            '*=', '-=', 'i++', 'j++', ';', 'select']

def create_features_from_text(X):
    X['body_first_len'] = X['BodyMarkdown'].apply(lambda x: len(list((re.findall(regex, x)) if re.findall(regex, x) else [''])[0]))
    X['title_sequences'] = tokenizer_title.texts_to_sequences(X['Title'].values)
    X['body_sequences'] = tokenizer_title.texts_to_sequences(X['BodyMarkdown'].values)
    X['title_len'] = X['Title'].apply(lambda x: len(x)).astype('float64')
    X['title_questions'] = X['BodyMarkdown'].apply(lambda x: sum(c == '?' for c in x)).astype('float64')
    X['body_questions'] = X['BodyMarkdown'].apply(lambda x: sum(c == '?' for c in x)).astype('float64')
    X['body_len'] = X['BodyMarkdown'].apply(lambda x: len(x)).astype('float64')
    X['body_title_len_diff'] = X['body_len'] - X['title_len']
    X['body_digits'] = X['BodyMarkdown'].apply(lambda x: sum(c.isdigit() for c in x)).astype('float64')
    X['body_capitals'] = X['BodyMarkdown'].apply(lambda x: sum(c.isupper() for c in x)).astype('float64')
    X['body_spaces'] = X['BodyMarkdown'].apply(lambda x: sum(c == '\r\n\r\n' for c in x)).astype('float64')
    X['body_when'] = X['BodyMarkdown'].apply(lambda x: x.lower().count('when')).astype('float64')
    X['title_when'] = X['Title'].apply(lambda x: x.lower().count('when')).astype('float64')
    X['body_error'] = X['BodyMarkdown'].apply(lambda x: x.lower().count('error')).astype('float64')
    X['body_exception'] = X['BodyMarkdown'].apply(lambda x: x.lower().count('exception')).astype('float64')
    X['body_best'] = X['BodyMarkdown'].apply(lambda x: x.lower().count('best')).astype('float64')
    X['title_best'] = X['Title'].apply(lambda x: x.lower().count('best')).astype('float64')
    X['body_good'] = X['BodyMarkdown'].apply(lambda x: x.lower().count('good')).astype('float64')
    X['title_good'] = X['Title'].apply(lambda x: x.lower().count('good')).astype('float64')
    X['body_please'] = X['BodyMarkdown'].apply(lambda x: x.lower().count('please')).astype('float64')
    X['title_please'] = X['Title'].apply(lambda x: x.lower().count('please')).astype('float64')
    X['body_wrong'] = X['BodyMarkdown'].apply(lambda x: x.lower().count('wrong')).astype('float64')
    X['body_sentences'] = X['BodyMarkdown'].apply(lambda x: len(list(re.finditer(regex, x))))
    X['body_I'] = X['BodyMarkdown'].apply(lambda x: x.count('I')).astype('float64')
    X['body_you'] = X['BodyMarkdown'].apply(lambda x: x.lower().count('you')).astype('float64')
    
    X['tags_weight'] = [float64(tags_weight(y)) for x, y in X.iterrows()] # adding sum of tags occurencies
    X['body_keywords'] = X['BodyMarkdown'].apply(lambda x: sum(c.lower() in keywords for c in x)).astype('float64')
    
    
create_features_from_text(Z_train)
create_features_from_text(Z_test)
# create_features_from_text(Z_emb)

/home/ihar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
/home/ihar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
/home/ihar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:52: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
/home/ihar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin i

In [14]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
    
tokenizer_tages = Tokenizer(filters='')
train_tags = Z_train[['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5']].astype('str').apply(lambda x: ' '.join(x), axis=1)
test_tags = Z_test[['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5']].astype('str').apply(lambda x: ' '.join(x), axis=1)

# tokenizer_tages.fit_on_texts(train_tags)
# tokenizer_tages.fit_on_texts(test_tags)

vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=0.01)
vectorizer.fit(train_tags)
vectorizer.fit(test_tags)

train_tags = vectorizer.transform(train_tags).toarray().tolist()
test_tags = vectorizer.transform(test_tags).toarray().tolist()

Z_train['tag_sequences'] = train_tags
# Z_emb['tag_sequences'] = tokenizer_tages.texts_to_sequences(train_tags)
Z_test['tag_sequences'] = test_tags

# lda = LatentDirichletAllocation(n_components=5, n_jobs=-1, learning_method='batch')
# lda.fit(train_tags)
# lda.fit(test_tags)

# Z_train['tags_lda'] = list(lda.transform(train_tags))
# # Z_emb['tags_lda'] = list(lda.transform(train_tags))
# Z_test['tags_lda'] = list(lda.transform(test_tags))

In [15]:
# BodyMarkdown
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=0.05)
vectorizer.fit(Z_train['BodyMarkdown'])
vectorizer.fit(Z_test['BodyMarkdown'])
# vectorizer.fit(Z_emb['BodyMarkdown'])

train_body = vectorizer.transform(Z_train['BodyMarkdown']).toarray().tolist()
test_body = vectorizer.transform(Z_test['BodyMarkdown']).toarray().tolist()
# emb_body = vectorizer.transform(Z_emb['BodyMarkdown'])

# lda = LatentDirichletAllocation(n_components=20, n_jobs=5, learning_method='batch')
# lda.fit(train_body)
# lda.fit(test_body)
# # lda.fit(emb_body)

# Z_train['body_lda'] = list(lda.transform(train_body))
# Z_test['body_lda'] = list(lda.transform(test_body))
# # Z_emb['body_lda'] = list(lda.transform(emb_body))

In [16]:
# Title
vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=0.05)
vectorizer.fit(Z_train['Title'])
vectorizer.fit(Z_test['Title'])
# vectorizer.fit(Z_emb['Title'])

train_titles = vectorizer.transform(Z_train['Title']).toarray().tolist()
# emb_titles = vectorizer.transform(Z_emb['Title'])
test_titles = vectorizer.transform(Z_test['Title']).toarray().tolist()

# lda = LatentDirichletAllocation(n_components=20, n_jobs=-1, learning_method='batch')
# lda.fit(train_titles)
# lda.fit(test_titles)
# # lda.fit(emb_titles)

# Z_train['title_lda'] = list(lda.transform(train_titles))
# # Z_emb['title_lda'] = list(lda.transform(emb_titles))
# Z_test['title_lda'] = list(lda.transform(test_titles))

In [17]:
from keras.preprocessing.sequence import pad_sequences

Z_train['title_sequences'] = train_titles # list(pad_sequences(Z_train['title_sequences'].values, maxlen=100))
Z_train['body_sequences'] = train_body # list(pad_sequences(Z_train['body_sequences'].values, maxlen=100))
# Z_emb['title_sequences'] = list(pad_sequences(Z_emb['title_sequences'].values, maxlen=100))
# Z_emb['body_sequences'] = list(pad_sequences(Z_emb['body_sequences'].values, maxlen=100))
Z_test['title_sequences'] = test_titles # list(pad_sequences(Z_test['title_sequences'].values, maxlen=100))
Z_test['body_sequences'] = test_body # list(pad_sequences(Z_test['body_sequences'].values, maxlen=100))

In [18]:
from sklearn import preprocessing
skip_columns = ['OpenStatus', 'PostCreationDate',
                                                 'PostClosedDate', 'OwnerUserId', 'OwnerCreationDate',
               'Title', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5', 'BodyMardown']

for column in list(Z_test.columns):
    if column in skip_columns:
        continue

    if not Z_test[column].dtype in ['float64', 'int64']:
        continue
        
    print(column)
    std_scale = preprocessing.StandardScaler().fit(Z_train[[column]])
    Z_train[column] = std_scale.transform([Z_train[column]])[0]
    Z_test[column] = std_scale.transform([Z_test[column]])[0]
    # Z_emb[column] = std_scale.transform([Z_emb[column]])[0]

ReputationAtPostCreation
OwnerUndeletedAnswerCountAtPostTime
body_first_len
title_len
title_questions
body_questions
body_len
body_title_len_diff
body_digits
body_capitals
body_spaces
body_when
title_when
body_error
body_exception
body_best
title_best
body_good
title_good
body_please
title_please


/home/ihar/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ihar/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ihar/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


body_wrong
body_sentences
body_I
body_you
tags_weight
body_keywords


/home/ihar/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [19]:
def create_date_features(X):
    X['creation_weekday'] = list(np_utils.to_categorical(X['PostCreationDate'].dt.dayofweek))
    X['creation_hour'] = list(np_utils.to_categorical(X['PostCreationDate'].dt.hour))
    X['creation_diff'] = (X['PostCreationDate'] - X['OwnerCreationDate']).dt.days
    
create_date_features(Z_test)
create_date_features(Z_train)
# create_date_features(Z_emb)

std_scale = preprocessing.StandardScaler().fit(Z_train[['creation_diff']])
Z_train['creation_diff'] = std_scale.transform([Z_train['creation_diff']])[0]
# Z_emb['creation_diff'] = std_scale.transform([Z_emb['creation_diff']])[0]
Z_test['creation_diff'] = std_scale.transform([Z_test['creation_diff']])[0]

/home/ihar/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [20]:
from sklearn import decomposition
def create_string_features(feature_list):
    for feature, pca_size in feature_list: 
        feaute_new = f'{feature}_word2vec'
        Z_train[feaute_new] = Z_train[feature].apply(lambda x: vectorise(x))
        Z_test[feaute_new] = Z_test[feature].apply(lambda x: vectorise(x))
        
#         pca = decomposition.PCA(n_components=pca_size)
#         pca.fit(list(Z_train[feaute_new].values))
#         pca.fit(list(Z_test[feaute_new].values))
        
#         Z_train[feaute_new] = list(pca.transform(list(Z_train[feaute_new].values)))
#         Z_test[feaute_new] = list(pca.transform(list(Z_test[feaute_new].values)))
    
create_string_features([('Title', 10), ('BodyMarkdown', 15), ('Tag1', 5), ('Tag2', 5), ('Tag3', 5), ('Tag4', 5), ('Tag5', 5)])

## Models

In [21]:
skip_columns_linear = ['OpenStatus', 'PostCreationDate',
                        'PostClosedDate', 'OwnerUserId', 'OwnerCreationDate',
                       'Title', 'BodyMarkdown', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5',
                        'Tag',
                       'body_lda', 'title_lda',]

# TOP features by XGB 
selected_features = ['BodyMarkdown_word2vec',
 'OwnerUndeletedAnswerCountAtPostTime',
 'ReputationAtPostCreation',
 'Tag1_word2vec',
 'Tag2_word2vec',
 'Tag3_word2vec',
 'Title_word2vec',
 'body_I',
 'body_capitals',
 'body_digits',
 'body_error',
 'body_keywords',
 'body_len',
 'body_sequences',
 'body_you',
 'creation_diff',
 'tag_sequences',
 'tags_weight',
 'title_best',
 'title_len',
 'title_questions']

X_train = Z_train.loc[:, [i for i in list(Z_train.columns) if i not in skip_columns_linear
                         and i in selected_features
                         ]].values
# X_emb = Z_train.loc[:, [i for i in list(Z_emb.columns) if i not in skip_columns_linear]].values
print(Z_train.loc[:, [i for i in list(Z_train.columns) if i not in skip_columns_linear
                    and i in selected_features
                     ]].columns)
Y_train = Z_train['OpenStatus'].astype('category')

X_test = Z_test.loc[:, [i for i in list(Z_test.columns) if i not in skip_columns_linear
                    and i in selected_features
                       ]].values
code_mapper = dict(enumerate(Y_train.cat.categories))
Y_train = Y_train.cat.codes

from keras.utils import np_utils
Y_train = np_utils.to_categorical(Y_train)

Index(['ReputationAtPostCreation', 'OwnerUndeletedAnswerCountAtPostTime', 'body_sequences', 'title_len', 'title_questions', 'body_len', 'body_digits', 'body_capitals', 'body_error', 'title_best', 'body_I', 'body_you', 'tags_weight', 'body_keywords', 'tag_sequences', 'creation_diff', 'Title_word2vec', 'BodyMarkdown_word2vec', 'Tag1_word2vec', 'Tag2_word2vec', 'Tag3_word2vec'], dtype='object')


In [22]:
X_train_sklearn = []
X_test_sklearn = []
X_emb_sklearn = []
for j in range(len(X_train)):
    X_train_sklearn.append(np.array(list(chain(*[X_train[j, i] if isinstance(X_train[j, i], (np.ndarray, list)) else [X_train[j, i]] for i in range(X_train.shape[1])]))))
    
# for j in range(len(X_train)):
#     X_emb_sklearn.append(np.array(list(chain(*[X_emb[j, i] if isinstance(X_emb[j, i], (np.ndarray, list)) else [X_train[j, i]] for i in range(X_emb.shape[1])]))))
    
for j in range(len(X_test)):
    X_test_sklearn.append(np.array(list(chain(*[X_test[j, i] if isinstance(X_test[j, i], (np.ndarray, list)) else [X_test[j, i]] for i in range(X_test.shape[1])]))))
    

In [23]:
X_train_sklearn = np.array(X_train_sklearn)
# X_emb_sklearn = np.array(X_emb_sklearn)
X_test_sklearn = np.array(X_test_sklearn)

In [24]:
X_train_sklearn.shape

(63462, 1813)

In [25]:
Y_train_sklearn = np.argmax(Y_train, axis=1)
Y_train_sklearn

array([0, 1, 0, 2, 0, 0, 3, 1, 2, 0, ..., 1, 3, 3, 0, 2, 3, 2, 3, 0, 2])

## KNN

In [31]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

kf, i, scores = KFold(n_splits=4), 0, []
for train, test in kf.split(X_train):
    knn = KNeighborsClassifier(n_neighbors=10, weights='distance', n_jobs=-1)
    knn.fit(X_train_sklearn[train], Y_train_sklearn[train])
    Y_pred = knn.predict(X_train_sklearn[test])
    
    score = accuracy_score(Y_train_sklearn[test], Y_pred)
    scores.append(score)
    i += 1
    print(f'Test result: {score}')

Test result: 0.517332661036178
Test result: 0.499495777133493
Test result: 0.5113142136779073
Test result: 0.505641348881185


## SGD

In [38]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf, i, scores = KFold(n_splits=4), 0, []
for train, test in kf.split(X_train):
    sgd = SGDClassifier(loss="log", penalty="l2", max_iter=60, n_jobs=-1)
    sgd.fit(X_train_sklearn[train], Y_train_sklearn[train])
    Y_pred = sgd.predict(X_train_sklearn[test])
    
    score = accuracy_score(Y_train_sklearn[test], Y_pred)
    scores.append(score)
    i += 1
    print(f'Test result: {score}')

Test result: 0.6193747636455313
Test result: 0.6081558048657507
Test result: 0.6229435865111882
Test result: 0.6187204538291837


In [39]:
print(np.mean(scores), np.std(scores))

0.6172986522129135 0.005517895870756234


In [42]:
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

kf = KFold(n_splits=4, shuffle=True)
scores = []
i = 0
for train, test in kf.split(X_train):
    model = RandomForestClassifier(n_estimators=190, max_depth=19, n_jobs=-1, class_weight='balanced',
                                  min_samples_leaf=7, max_features=275)
# params = {
#     'n_estimators': np.linspace(10, 150, 15),
#     'max_depth': list(range(10, 20)),
#     'n_jobs': [-1],
#     'class_weight': ['balanced', None],
#     'max_features': ['auto', 'sqrt', 'log2', None],
#     'min_samples_leaf': list(range(10, 100)),
# }
    model.fit(X_train_sklearn[train], Y_train_sklearn[train])
    Y_hat = model.predict(X_train_sklearn[test])
    
    score = accuracy_score(Y_train_sklearn[test], Y_hat)
    scores.append(score)
    i += 1
    print(f'Test result: {score}')



Test result: 0.6000252111433253
Test result: 0.599457960418505
Test result: 0.5974787267570123
Test result: 0.6027103687362118


In [43]:
print(np.mean(scores), np.std(scores))

0.5999180667637636 0.0018688571200665795


## Train XGBoost and seletc top features

In [135]:
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split



kf = KFold(n_splits=10, shuffle=True)
scores = []
scores0 = []
scores1 = []
scores2 = []
scores4 = []
i = 0
Y_hats = []
Y_hats1 = []
Y_hats2 = []
dtest_all = xgb.DMatrix(X_test_sklearn, nthread=-1, silent=False)
for train, test in kf.split(X_train):
    train, val = train_test_split(train, test_size=0.1)
    
    dtrain = xgb.DMatrix(X_train_sklearn[train], label=Y_train_sklearn[train], nthread=-1, silent=False)
    dtrain.save_binary('train.xgb')
    dtest = xgb.DMatrix(X_train_sklearn[test], label=Y_train_sklearn[test], nthread=-1, silent=False)
    dtrain.save_binary('val.xgb')
    dval = xgb.DMatrix(X_train_sklearn[val], label=Y_train_sklearn[val], nthread=-1, silent=False)
    dtrain.save_binary('test.xgb')
    param = {'max_depth': 3, 'eta': 0.1, 'redictor': 'gpu_predictor', 'silent': 0,
             'tree_method': 'gpu_hist', 'alpha': 0.01, 
            # 'evallist': [(dval, 'eval'), (dtrain, 'train')],
            'objective': 'multi:softprob', 'eval_metric': 'merror',
            'num_class': 5}
    model = xgb.train(param, dtrain, 700, early_stopping_rounds=45, evals=[(dtrain, 'train'),(dval, 'eval')])
    Y_hat = model.predict(dtest, ntree_limit=model.best_ntree_limit)
    save_model_name = f'xgb_6_{i}'
    model.save_model(save_model_name)
    print(model.best_score)
    
    score = accuracy_score(Y_train_sklearn[test], Y_hat.argmax(axis=1))
    scores.append(score)
    print(f'Single model: {score}') 
    
    
    Y_hat = model.predict(dtest_all, ntree_limit=int(model.attributes()['best_iteration']) + 1)
    Y_hats.append(Y_hat)
    # del model
    i += 1


[0]	train-merror:0.488026	eval-merror:0.486169
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 45 rounds.
[1]	train-merror:0.47507	eval-merror:0.471464
[2]	train-merror:0.471451	eval-merror:0.467437
[3]	train-merror:0.464117	eval-merror:0.458508
[4]	train-merror:0.460304	eval-merror:0.453782
[5]	train-merror:0.455752	eval-merror:0.450105
[6]	train-merror:0.452075	eval-merror:0.447129
[7]	train-merror:0.449079	eval-merror:0.445203
[8]	train-merror:0.446453	eval-merror:0.443277
[9]	train-merror:0.443379	eval-merror:0.43855
[10]	train-merror:0.441589	eval-merror:0.438726
[11]	train-merror:0.438632	eval-merror:0.438375
[12]	train-merror:0.437134	eval-merror:0.4375
[13]	train-merror:0.43585	eval-merror:0.4375
[14]	train-merror:0.434138	eval-merror:0.435924
[15]	train-merror:0.431687	eval-merror:0.433298
[16]	train-merror:0.429625	eval-merror:0.432948
[17]	train-merror:0.428282	eval-merror:0.430322
[18]	t

[168]	train-merror:0.336868	eval-merror:0.366246
[169]	train-merror:0.336692	eval-merror:0.365896
[170]	train-merror:0.336381	eval-merror:0.366422
[171]	train-merror:0.336245	eval-merror:0.366422
[172]	train-merror:0.336128	eval-merror:0.366597
[173]	train-merror:0.335681	eval-merror:0.366597
[174]	train-merror:0.33535	eval-merror:0.366246
[175]	train-merror:0.335292	eval-merror:0.366422
[176]	train-merror:0.334942	eval-merror:0.366246
[177]	train-merror:0.334883	eval-merror:0.366597
[178]	train-merror:0.334669	eval-merror:0.367472
[179]	train-merror:0.334514	eval-merror:0.367122
[180]	train-merror:0.334008	eval-merror:0.367472
[181]	train-merror:0.33356	eval-merror:0.367472
[182]	train-merror:0.333463	eval-merror:0.367122
[183]	train-merror:0.332763	eval-merror:0.367122
[184]	train-merror:0.332704	eval-merror:0.367647
[185]	train-merror:0.332841	eval-merror:0.367647
[186]	train-merror:0.332354	eval-merror:0.367997
[187]	train-merror:0.332199	eval-merror:0.367122
[188]	train-merror:0.3

[15]	train-merror:0.432056	eval-merror:0.443803
[16]	train-merror:0.43015	eval-merror:0.443452
[17]	train-merror:0.429158	eval-merror:0.440651
[18]	train-merror:0.426629	eval-merror:0.440301
[19]	train-merror:0.424606	eval-merror:0.439076
[20]	train-merror:0.423458	eval-merror:0.436975
[21]	train-merror:0.422466	eval-merror:0.438025
[22]	train-merror:0.419937	eval-merror:0.434349
[23]	train-merror:0.418886	eval-merror:0.433824
[24]	train-merror:0.416513	eval-merror:0.431548
[25]	train-merror:0.41552	eval-merror:0.430672
[26]	train-merror:0.413964	eval-merror:0.428746
[27]	train-merror:0.412661	eval-merror:0.427521
[28]	train-merror:0.411007	eval-merror:0.426471
[29]	train-merror:0.409801	eval-merror:0.423845
[30]	train-merror:0.408634	eval-merror:0.42437
[31]	train-merror:0.407311	eval-merror:0.42437
[32]	train-merror:0.405307	eval-merror:0.422619
[33]	train-merror:0.404159	eval-merror:0.422794
[34]	train-merror:0.403478	eval-merror:0.421218
[35]	train-merror:0.401903	eval-merror:0.420

[185]	train-merror:0.331693	eval-merror:0.366597
[186]	train-merror:0.331479	eval-merror:0.366597
[187]	train-merror:0.331148	eval-merror:0.366246
[188]	train-merror:0.330798	eval-merror:0.364671
[189]	train-merror:0.33072	eval-merror:0.364846
[190]	train-merror:0.330311	eval-merror:0.366071
[191]	train-merror:0.33	eval-merror:0.365196
[192]	train-merror:0.329494	eval-merror:0.366071
[193]	train-merror:0.329397	eval-merror:0.365721
[194]	train-merror:0.329125	eval-merror:0.366772
[195]	train-merror:0.329066	eval-merror:0.366071
[196]	train-merror:0.328852	eval-merror:0.365371
[197]	train-merror:0.32895	eval-merror:0.364671
[198]	train-merror:0.328308	eval-merror:0.365021
[199]	train-merror:0.3286	eval-merror:0.365721
[200]	train-merror:0.328269	eval-merror:0.365196
[201]	train-merror:0.327958	eval-merror:0.365021
[202]	train-merror:0.327977	eval-merror:0.364846
[203]	train-merror:0.327296	eval-merror:0.364671
[204]	train-merror:0.327335	eval-merror:0.365371
[205]	train-merror:0.327432	

Will train until eval-merror hasn't improved in 45 rounds.
[1]	train-merror:0.474282	eval-merror:0.475665
[2]	train-merror:0.467279	eval-merror:0.466387
[3]	train-merror:0.460178	eval-merror:0.459909
[4]	train-merror:0.457999	eval-merror:0.454832
[5]	train-merror:0.454439	eval-merror:0.455532
[6]	train-merror:0.451113	eval-merror:0.453081
[7]	train-merror:0.449459	eval-merror:0.453256
[8]	train-merror:0.445782	eval-merror:0.44993
[9]	train-merror:0.443779	eval-merror:0.45063
[10]	train-merror:0.440938	eval-merror:0.445903
[11]	train-merror:0.438001	eval-merror:0.444853
[12]	train-merror:0.436639	eval-merror:0.443452
[13]	train-merror:0.433877	eval-merror:0.442927
[14]	train-merror:0.433371	eval-merror:0.441702
[15]	train-merror:0.431056	eval-merror:0.440651
[16]	train-merror:0.42878	eval-merror:0.439601
[17]	train-merror:0.426815	eval-merror:0.437675
[18]	train-merror:0.42487	eval-merror:0.4368
[19]	train-merror:0.422866	eval-merror:0.434699
[20]	train-merror:0.421251	eval-merror:0.433

...

[71]	train-merror:0.371819	eval-merror:0.395483
[72]	train-merror:0.371158	eval-merror:0.396534
[73]	train-merror:0.370983	eval-merror:0.396359
[74]	train-merror:0.370438	eval-merror:0.395483
[75]	train-merror:0.369232	eval-merror:0.395483
[76]	train-merror:0.369037	eval-merror:0.394783
[77]	train-merror:0.368201	eval-merror:0.394258
[78]	train-merror:0.367948	eval-merror:0.394258
[79]	train-merror:0.367326	eval-merror:0.392507
[80]	train-merror:0.366625	eval-merror:0.392857
[81]	train-merror:0.3661	eval-merror:0.393382
[82]	train-merror:0.365516	eval-merror:0.392157
[83]	train-merror:0.364816	eval-merror:0.391282
[84]	train-merror:0.364544	eval-merror:0.391282
[85]	train-merror:0.363668	eval-merror:0.391457
[86]	train-merror:0.363201	eval-merror:0.392157
[87]	train-merror:0.362793	eval-merror:0.390231
[88]	train-merror:0.362054	eval-merror:0.390406
[89]	train-merror:0.362112	eval-merror:0.390406
[90]	train-merror:0.361762	eval-merror:0.390406
[91]	train-merror:0.361081	eval-merror:0.3

[240]	train-merror:0.318808	eval-merror:0.365021
[241]	train-merror:0.318711	eval-merror:0.365021
[242]	train-merror:0.318458	eval-merror:0.364671
[243]	train-merror:0.318166	eval-merror:0.364321
[244]	train-merror:0.317757	eval-merror:0.364846
[245]	train-merror:0.317407	eval-merror:0.365546
[246]	train-merror:0.317135	eval-merror:0.365721
[247]	train-merror:0.317057	eval-merror:0.365721
[248]	train-merror:0.317115	eval-merror:0.365896
[249]	train-merror:0.316862	eval-merror:0.365546
[250]	train-merror:0.316629	eval-merror:0.365196
[251]	train-merror:0.316473	eval-merror:0.365721
[252]	train-merror:0.316201	eval-merror:0.365371
[253]	train-merror:0.316396	eval-merror:0.365196
[254]	train-merror:0.316104	eval-merror:0.365546
[255]	train-merror:0.316045	eval-merror:0.365546
[256]	train-merror:0.315909	eval-merror:0.365196
[257]	train-merror:0.31554	eval-merror:0.365896
[258]	train-merror:0.315248	eval-merror:0.365546
[259]	train-merror:0.315189	eval-merror:0.365546
[260]	train-merror:0.

[27]	train-merror:0.410104	eval-merror:0.413165
[28]	train-merror:0.408723	eval-merror:0.410364
[29]	train-merror:0.407031	eval-merror:0.41124
[30]	train-merror:0.405649	eval-merror:0.41159
[31]	train-merror:0.404716	eval-merror:0.408964
[32]	train-merror:0.403918	eval-merror:0.408613
[33]	train-merror:0.402148	eval-merror:0.407738
[34]	train-merror:0.401973	eval-merror:0.407038
[35]	train-merror:0.400728	eval-merror:0.404062
[36]	train-merror:0.399911	eval-merror:0.405287
[37]	train-merror:0.398879	eval-merror:0.404762
[38]	train-merror:0.397615	eval-merror:0.404762
[39]	train-merror:0.397226	eval-merror:0.403887
[40]	train-merror:0.396292	eval-merror:0.40056
[41]	train-merror:0.39458	eval-merror:0.401085
[42]	train-merror:0.393607	eval-merror:0.401961
[43]	train-merror:0.392285	eval-merror:0.401085
[44]	train-merror:0.391643	eval-merror:0.400035
[45]	train-merror:0.390806	eval-merror:0.39986
[46]	train-merror:0.390047	eval-merror:0.398284
[47]	train-merror:0.389522	eval-merror:0.3972

[197]	train-merror:0.328671	eval-merror:0.359944
[198]	train-merror:0.328632	eval-merror:0.359594
[199]	train-merror:0.328399	eval-merror:0.359769
[200]	train-merror:0.328146	eval-merror:0.360294
[201]	train-merror:0.328068	eval-merror:0.359769
[202]	train-merror:0.327679	eval-merror:0.359594
[203]	train-merror:0.327718	eval-merror:0.359244
[204]	train-merror:0.327426	eval-merror:0.359944
[205]	train-merror:0.327192	eval-merror:0.359769
[206]	train-merror:0.32694	eval-merror:0.358543
[207]	train-merror:0.326726	eval-merror:0.358368
[208]	train-merror:0.326298	eval-merror:0.358193
[209]	train-merror:0.326259	eval-merror:0.357668
[210]	train-merror:0.326181	eval-merror:0.358193
[211]	train-merror:0.325811	eval-merror:0.357668
[212]	train-merror:0.325947	eval-merror:0.357843
[213]	train-merror:0.325636	eval-merror:0.357493
[214]	train-merror:0.325344	eval-merror:0.357668
[215]	train-merror:0.324975	eval-merror:0.357493
[216]	train-merror:0.324644	eval-merror:0.358368
[217]	train-merror:0.

[365]	train-merror:0.295347	eval-merror:0.35049
[366]	train-merror:0.295035	eval-merror:0.350315
[367]	train-merror:0.294938	eval-merror:0.351015
[368]	train-merror:0.294588	eval-merror:0.351015
[369]	train-merror:0.294335	eval-merror:0.351541
[370]	train-merror:0.29416	eval-merror:0.35119
[371]	train-merror:0.294141	eval-merror:0.352066
[372]	train-merror:0.293713	eval-merror:0.351891
[373]	train-merror:0.293674	eval-merror:0.351541
[374]	train-merror:0.293401	eval-merror:0.351716
[375]	train-merror:0.293168	eval-merror:0.351891
[376]	train-merror:0.293071	eval-merror:0.351541
[377]	train-merror:0.292857	eval-merror:0.351366
[378]	train-merror:0.292934	eval-merror:0.35119
[379]	train-merror:0.292331	eval-merror:0.351716
[380]	train-merror:0.292195	eval-merror:0.35119
[381]	train-merror:0.292098	eval-merror:0.350665
[382]	train-merror:0.291942	eval-merror:0.350665
[383]	train-merror:0.291689	eval-merror:0.351015
[384]	train-merror:0.291436	eval-merror:0.351366
[385]	train-merror:0.2916

In [35]:
print(np.mean(scores), np.std(scores))

0.6359396884586949 0.004144995966019183


In [120]:
import operator
from collections import OrderedDict
features = model.get_score(importance_type='weight')
feature_important = OrderedDict(sorted(features.items(), key=operator.itemgetter(1), reverse=True))

keys = list(feature_important.keys())
values = list(feature_important.values())

In [121]:
def get_feature_number(x):
    sum_len = 0
    i = 0
    for column in Z_train.loc[:, [i for i in list(Z_train.columns) if i not in skip_columns_linear
                                  and i in selected_features
                                 ]]\
        .columns:
        try:
            sum_len += np.array(Z_train[column].values[0]).shape[0]
        except:
            sum_len += 1
            
        if sum_len > x:
            break
        i += 1
    return i

set(Z_train.loc[:, [i for i in list(Z_train.columns) if i not in skip_columns_linear
                and i in selected_features
               ]]\
.columns[[get_feature_number(int(key.replace('f', ''))) for key in keys[:200]]])

{'BodyMarkdown_word2vec',
 'OwnerUndeletedAnswerCountAtPostTime',
 'ReputationAtPostCreation',
 'Tag1_word2vec',
 'Title_word2vec',
 'body_capitals',
 'body_digits',
 'body_keywords',
 'body_len',
 'body_sequences',
 'creation_diff',
 'tags_weight',
 'title_best',
 'title_len',
 'title_questions'}

In [27]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb

In [28]:
#Validation function
n_folds = 5
from sklearn.metrics.scorer import make_scorer

def metric(y_pred, y):
    if isinstance(y[0], np.ndarray):
        return accuracy_score(y_pred, y.argmax(axis=1))
    elif isinstance(y_pred[0], np.ndarray):
        return accuracy_score(y_pred.argmax(axis=1), y)
    else:
        return accuracy_score(y_pred, y)

my_scorer = make_scorer(metric, greater_is_better=True)

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True).get_n_splits(X_train_sklearn)
    rmse= cross_val_score(model, X_train_sklearn, Y_train_sklearn, scoring=my_scorer, cv = kf)
    return(rmse)

## XGBoost

In [39]:
from sklearn.base import BaseEstimator, ClassifierMixin

class XGBClassifier(BaseEstimator):
    def fit(self, X, y=None):
        # x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
        dtrain = xgb.DMatrix(X, label=y, nthread=-1, silent=False)
        dtrain.save_binary('train.xgb')
#         dval = xgb.DMatrix(x_val, label=y_val, nthread=-1, silent=False)
#         dval.save_binary('val.xgb')
        param = {'max_depth': 2, 'eta': 0.3, 'redictor': 'gpu_predictor', 'silent': 0,
                 # 'tree_method': 'gpu_hist',
                 'alpha': 0.01, 'grow_policy': 'lossguide',
                 'max_leaves': 5, 'gamma': 0.02,
                 'colsample_bylevel': 0.5,
                'objective': 'multi:softprob', 'eval_metric': 'merror',
                 'nthread': 7,
                 'subsample': 0.8,
                'num_class': 5}
        self.model = xgb.train(param, dtrain, 190, 
        #                       early_stopping_rounds=45, evals=[(dtrain, 'train'),(dval, 'eval')]
                              )

    def predict(self, X, y=None):
        dtest = xgb.DMatrix(X, nthread=-1, silent=False)
        dtest.save_binary('test.xgb')
        return self.model.predict(dtest, ntree_limit=self.model.best_ntree_limit)
            

In [37]:
score = rmsle_cv(XGBClassifier())
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Gradient Boosting score: 0.6376 (0.0035)



## LGBM

In [40]:

lgbm_params =  {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 5,
    'metric': ['multi_error'],
    "learning_rate": 0.05,
     "num_leaves": 10,
     "max_depth": 5,
     "feature_fraction": 0.9,
     "bagging_fraction": 0.5,
     "reg_alpha": 0.05,
     "reg_lambda": 0.05,
#      "min_split_gain": 0,
      "min_child_weight": 0,
    # 'min_data_in_leaf'
        'nthread': 6,
}

class LGBMClassifier(BaseEstimator):
    def fit(self, X, y=None):
        # x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
        dtrain = lgb.Dataset(X, label=y)
        dtrain.save_binary('train.lgb')
#         dval = lgb.Dataset(x_val, label=y_val, reference=dtrain)
#         dval.save_binary('val.lgb')
        self.model = lgb.train(lgbm_params, dtrain, 350,
                               # early_stopping_rounds=25, valid_sets=[dtrain, dval]
                              )
        
    def predict(self, X, y=None):
#         dtest = lgb.Dataset(X)
#         dtest.save_binary('test.lgb')
        return self.model.predict(X, num_iteration=self.model.best_iteration)



In [48]:
score = rmsle_cv(LGBMClassifier())
print("LGBM score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

LGBM score: 0.6388 (0.0044)



## Neural Network

In [34]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, BatchNormalization,\
    Dropout, Conv1D, Embedding, Reshape, Flatten, LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import keras

activation = 'selu'
def create_model():
    keras.backend.clear_session()
    
    input_all = Input(shape=(1813, ))
    
    drop = Dropout(0.38)(input_all)
    dense = Dense(23, activation=activation)(drop)
    dense = BatchNormalization()(dense)
    dense = Dense(5, activation='softmax')(dense)
    model = Model(inputs=input_all, outputs=dense)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

create_model().summary()
class NNClassifier(BaseEstimator):
    def fit(self, X, y=None):
        self.model = create_model()
        save_model_name = f'nn'
        batch_size = 512
        
        y_cat = np_utils.to_categorical(y)
        model_checkpoint = ModelCheckpoint(save_model_name, monitor='val_loss',
                                           save_best_only=True)
#         reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=5, min_lr=0.001)
        self.model.fit(X, y_cat, verbose=1, epochs=40,
#                 validation_split=0.1,  callbacks=[model_checkpoint],
                       batch_size=batch_size)
        # self.model.load_weights(filepath=save_model_name)
        
    def predict(self, X, y=None):
#         dtest = lgb.Dataset(X)
#         dtest.save_binary('test.lgb')
        return self.model.predict(X)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1813)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1813)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 23)                41722     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23)                92        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 120       
Total params: 41,934
Trainable params: 41,888
Non-trainable params: 46
_________________________________________________________________


In [35]:
score = rmsle_cv(NNClassifier())
print("NN score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Epoch 1/40
50769/50769 [==============================] - 1s 11us/step - loss: 1.2990 - acc: 0.5132
Epoch 2/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0939 - acc: 0.5876
Epoch 3/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0556 - acc: 0.5962
Epoch 4/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0398 - acc: 0.6005
Epoch 5/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0277 - acc: 0.6031
Epoch 6/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0249 - acc: 0.6040
Epoch 7/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0196 - acc: 0.6056
Epoch 8/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0157 - acc: 0.6083
Epoch 9/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0159 - acc: 0.6084
Epoch 10/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0108 - acc: 0.6112
Epoch 11

50770/50770 [==============================] - 0s 7us/step - loss: 1.0508 - acc: 0.5980
Epoch 4/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0356 - acc: 0.6024
Epoch 5/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0268 - acc: 0.6047
Epoch 6/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0221 - acc: 0.6076
Epoch 7/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0167 - acc: 0.6078
Epoch 8/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0157 - acc: 0.6088
Epoch 9/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0138 - acc: 0.6085
Epoch 10/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0078 - acc: 0.6118
Epoch 11/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0069 - acc: 0.6118
Epoch 12/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0026 - acc: 0.6141
Epoch 13/40
50770/

50770/50770 [==============================] - 0s 8us/step - loss: 1.0264 - acc: 0.6038
Epoch 6/40
50770/50770 [==============================] - 0s 8us/step - loss: 1.0207 - acc: 0.6067
Epoch 7/40
50770/50770 [==============================] - 0s 8us/step - loss: 1.0181 - acc: 0.6083
Epoch 8/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0142 - acc: 0.6096
Epoch 9/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0101 - acc: 0.6110
Epoch 10/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0107 - acc: 0.6094
Epoch 11/40
50770/50770 [==============================] - 0s 7us/step - loss: 1.0079 - acc: 0.6103
Epoch 12/40
50770/50770 [==============================] - 0s 8us/step - loss: 1.0036 - acc: 0.6149
Epoch 13/40
50770/50770 [==============================] - 0s 8us/step - loss: 1.0030 - acc: 0.6130
Epoch 14/40
50770/50770 [==============================] - 0s 8us/step - loss: 1.0000 - acc: 0.6131
Epoch 15/40
5077

In [36]:
print("NN score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

NN score: 0.6328 (0.0053)



In [37]:
class AveragingModels(BaseEstimator):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = [
            model.predict(X) for model in self.models_
        ]
        return np.mean(predictions, axis=0)   

In [41]:
averaged_models = AveragingModels(models = (NNClassifier(), XGBClassifier(), LGBMClassifier()))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Epoch 1/40
50769/50769 [==============================] - 0s 9us/step - loss: 1.2910 - acc: 0.5084
Epoch 2/40
50769/50769 [==============================] - 0s 8us/step - loss: 1.0906 - acc: 0.5878
Epoch 3/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0519 - acc: 0.5950
Epoch 4/40
50769/50769 [==============================] - 0s 8us/step - loss: 1.0354 - acc: 0.6010
Epoch 5/40
50769/50769 [==============================] - 0s 8us/step - loss: 1.0271 - acc: 0.6051
Epoch 6/40
50769/50769 [==============================] - 0s 8us/step - loss: 1.0236 - acc: 0.6051
Epoch 7/40
50769/50769 [==============================] - 0s 8us/step - loss: 1.0155 - acc: 0.6075
Epoch 8/40
50769/50769 [==============================] - 0s 8us/step - loss: 1.0139 - acc: 0.6081
Epoch 9/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0095 - acc: 0.6098
Epoch 10/40
50769/50769 [==============================] - 0s 7us/step - loss: 1.0087 - acc: 0.6111
Epoch 11/

[22:28:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:35] /workspace/sr

[22:28:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:44] /workspace/sr

[22:28:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:52] /workspace/sr

[22:28:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:28:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:29:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:29:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:29:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:29:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:29:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:29:00] /workspace/sr

...

[22:44:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:44:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:44:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:44:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:44:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:44:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:44:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:44:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:44:53] /workspace/sr

[22:45:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:01] /workspace/sr

[22:45:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:10] /workspace/sr

[22:45:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:18] /workspace/sr

[22:45:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:45:27] /workspace/sr

In [42]:
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.6455 (0.0059)



In [43]:
averaged_models = AveragingModels(models = (NNClassifier(), XGBClassifier(), LGBMClassifier()))
averaged_models.fit(X_train_sklearn, Y_train_sklearn)
averaged_models_pred = averaged_models.predict(X_test_sklearn)

Train on 57115 samples, validate on 6347 samples
Epoch 1/200
57115/57115 [==============================] - 1s 23us/step - loss: 1.2073 - acc: 0.5361 - val_loss: 1.0192 - val_acc: 0.6153
Epoch 2/200
57115/57115 [==============================] - 1s 20us/step - loss: 1.0694 - acc: 0.5855 - val_loss: 1.0088 - val_acc: 0.6260
Epoch 3/200
57115/57115 [==============================] - 1s 20us/step - loss: 1.0567 - acc: 0.5905 - val_loss: 1.0004 - val_acc: 0.6230
Epoch 4/200
57115/57115 [==============================] - 1s 20us/step - loss: 1.0464 - acc: 0.5947 - val_loss: 0.9979 - val_acc: 0.6231
Epoch 5/200
57115/57115 [==============================] - 1s 21us/step - loss: 1.0430 - acc: 0.5942 - val_loss: 0.9934 - val_acc: 0.6271
Epoch 6/200
57115/57115 [==============================] - 1s 20us/step - loss: 1.0391 - acc: 0.5962 - val_loss: 0.9935 - val_acc: 0.6230
Epoch 7/200
57115/57115 [==============================] - 1s 20us/step - loss: 1.0310 - acc: 0.6002 - val_loss: 0.9861 - v

Epoch 118/200
57115/57115 [==============================] - 1s 20us/step - loss: 0.9608 - acc: 0.6286 - val_loss: 0.9494 - val_acc: 0.6409
Epoch 119/200
57115/57115 [==============================] - 1s 21us/step - loss: 0.9610 - acc: 0.6294 - val_loss: 0.9546 - val_acc: 0.6389
Epoch 120/200
57115/57115 [==============================] - 1s 21us/step - loss: 0.9626 - acc: 0.6282 - val_loss: 0.9481 - val_acc: 0.6464
Epoch 121/200
57115/57115 [==============================] - 1s 21us/step - loss: 0.9615 - acc: 0.6291 - val_loss: 0.9498 - val_acc: 0.6411
Epoch 122/200
57115/57115 [==============================] - 1s 20us/step - loss: 0.9599 - acc: 0.6280 - val_loss: 0.9492 - val_acc: 0.6403
Epoch 123/200
57115/57115 [==============================] - 1s 20us/step - loss: 0.9609 - acc: 0.6285 - val_loss: 0.9515 - val_acc: 0.6386
Epoch 124/200
57115/57115 [==============================] - 1s 20us/step - loss: 0.9607 - acc: 0.6282 - val_loss: 0.9498 - val_acc: 0.6398
Epoch 125/200
57115/

[18:33:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:33:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:33:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:33:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:33:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[7]	train-merror:0.448691	eval-merror:0.452182
[18:33:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:33:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:33:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra node

[18:34:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19]	train-merror:0.42523	eval-merror:0.42587
[18:34:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes

[18:34:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[31]	train-merror:0.407284	eval-merror:0.411533
[18:34:51] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[32]	train-merror:0.406391	eval-merror:0.4098
[18:34:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:34:54] /workspace/src/tree/updater_prune.

...

[18:43:38] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[279]	train-merror:0.31321	eval-merror:0.357177
[18:43:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:43:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:43:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:43:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:43:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[280]	train-merror:0.313018	eval-merror:0.356862
[18:43:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:43:41] /workspace/src/tree/updater_prun

[18:44:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:05] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[292]	train-merror:0.311004	eval-merror:0.356389
[18:44:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra no

[18:44:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[304]	train-merror:0.309166	eval-merror:0.355759
[18:44:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:44:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[305]	train-merror:0.309026	eval-merror:0.355759
[18:44:33] /workspace/src/tree/updater_pr

[86]	valid_0's multi_error: 0.401134
[87]	valid_0's multi_error: 0.400189
[88]	valid_0's multi_error: 0.400032
[89]	valid_0's multi_error: 0.400032
[90]	valid_0's multi_error: 0.400977
[91]	valid_0's multi_error: 0.400189
[92]	valid_0's multi_error: 0.398771
[93]	valid_0's multi_error: 0.400189
[94]	valid_0's multi_error: 0.399244
[95]	valid_0's multi_error: 0.397983
[96]	valid_0's multi_error: 0.398614
[97]	valid_0's multi_error: 0.398929
[98]	valid_0's multi_error: 0.398614
[99]	valid_0's multi_error: 0.397826
[100]	valid_0's multi_error: 0.397668
[101]	valid_0's multi_error: 0.397038
[102]	valid_0's multi_error: 0.397196
[103]	valid_0's multi_error: 0.397826
[104]	valid_0's multi_error: 0.396723
[105]	valid_0's multi_error: 0.39625
[106]	valid_0's multi_error: 0.396723
[107]	valid_0's multi_error: 0.396723
[108]	valid_0's multi_error: 0.39688
[109]	valid_0's multi_error: 0.395778
[110]	valid_0's multi_error: 0.396093
[111]	valid_0's multi_error: 0.396093
[112]	valid_0's multi_error:

[304]	valid_0's multi_error: 0.370096
[305]	valid_0's multi_error: 0.370569
[306]	valid_0's multi_error: 0.370096
[307]	valid_0's multi_error: 0.370411
[308]	valid_0's multi_error: 0.370254
[309]	valid_0's multi_error: 0.370096
[310]	valid_0's multi_error: 0.370411
[311]	valid_0's multi_error: 0.370411
[312]	valid_0's multi_error: 0.370411
[313]	valid_0's multi_error: 0.369939
[314]	valid_0's multi_error: 0.369308
[315]	valid_0's multi_error: 0.370096
[316]	valid_0's multi_error: 0.369308
[317]	valid_0's multi_error: 0.369308
[318]	valid_0's multi_error: 0.369308
[319]	valid_0's multi_error: 0.369623
[320]	valid_0's multi_error: 0.368836
[321]	valid_0's multi_error: 0.369151
[322]	valid_0's multi_error: 0.368993
[323]	valid_0's multi_error: 0.369466
[324]	valid_0's multi_error: 0.368678
[325]	valid_0's multi_error: 0.368678
[326]	valid_0's multi_error: 0.369151
[327]	valid_0's multi_error: 0.369151
[328]	valid_0's multi_error: 0.368836
[329]	valid_0's multi_error: 0.368836
[330]	valid_

In [46]:
accuracy_score(Y_train_sklearn, averaged_models_pred.argmax(axis=1))

0.645867

In [48]:
Y_subm = [code_mapper[np.argmax(y)] for y in averaged_models_pred]

# Submission

In [49]:
S = pd.DataFrame(Y_subm, columns=['OpenStatus'], index = Z_test.index)

In [50]:
S.head()

,OpenStatus
PostId,
2,open
4,open
8,not a real question
10,open
11,not a real question


In [51]:
S

,OpenStatus
PostId,
2,open
4,open
8,not a real question
10,open
11,not a real question
13,off topic
16,not a real question
20,open
21,not constructive


In [52]:
S.to_csv('solution_28.csv')